# C3S_422_Lot2 Plymouth Marine Laboratory: Marine, Coastal and Fisheries Sectoral Information Systems. #
# Natural Capital Accounting Demonstrator tutorial.  #

## Introduction
This notebook provides instructions on how to access climate modelling and fisheries projections for natural capital accounting.  

The data used for processing and visualisation will be the outputs of the coupled physical biogeochemical model ERSEM and its Tier I indicators  for temperature, salinity, pH; and fisheries potential catch projections from the SS-DBEM as Tier 2 indicators. The valuation of the potential catch is carried out using fish price data providing a Tier 3 indicator. 

The presented method will require manipulating model simulations, analysis of the selected projections, valuation of potential fish catch and presentation of the results both as maps and tables. Each natural capital account is now taken in turn and details provided.

First we have to import the Python packages used in the processing. This may take a couple of minutes.

In [1]:
import cdsapi
import numpy as np
import xarray as xr
import geoviews as gv
import geoviews.feature as gf
from geoviews.operation.regrid import weighted_regrid
from cartopy import crs
import cartopy.feature as cfeature
import holoviews as hv
from holoviews.operation.datashader import regrid
import zipfile as zip
import tempfile
import re
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from cartopy.io import shapereader

import pandas as pd
import zipfile as zip

and set up some options for our visualisations.

In [2]:
hv.notebook_extension('matplotlib')

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [3]:
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [4]:
%opts Image {+framewise} [colorbar=True, ] Curve [xrotation=60]
%output max_frames=100000

In [5]:
land_10m = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='black',
                                        facecolor='#C0E8C0')

## a) Physical asset account
### Example application: 
Map the average temperature, salinity and pH by gridded cell within the UK’s EEZ waters (48.4° to 63.5°N and 10.8°W to 3.2°E) under the climate change RCP 4.5 for the years 2013-2017 and 2050. Calculate and record the average value, as well as minimum & maximum values, for each element for the whole of the EEZ in 2013-2017 and 2050 under the RCP4.5 and present in a table. 

### Input data:
*.netCDF file of ERSEM monthly outputs for RCP 4.5 for temperature, salinity and pH in the domain 48.4° to 63.5°N and 10.8°W to 3.2°E.

### Output data: 
Average values for each element for the years 2013-2017 and 2050 by gridded cell and aggregated averages as well as minimum & maximum vales for the whole EEZ for each of the years. 

### Python script: 
Extracts the monthly levels of temperature, salinity and pH for each of the years 2013-2017 and 2050 for the UK EEZ waters (48.4° to 63.5°N and 10.8°W to 3.2°E) and calculates the average per gridded cell. The results are then presented as spatially resolved maps and downloadable as a jpeg or other appropriate format.  Aggregated averages as well as minimum & maximum levels for the whole EEZ for each of the years are also calculated. Outputs are presented as tables and downloadable as a csv file. 

We will locate and download the POLCOMS-ERSEM date from the CDS (see https://github.com/pmlrsg/c3smcf/blob/master/notebooks/01_Introduction-Accessing-MCF-data-in-the-CDS.ipynb for more details). We will also download the SS-DBEM data which is used later as we wish to map the ERSEM data on the same grid as the SS-DBEM. This is the lower resolution of the two (0.5 deg x 0.5 deg) and we will downsample the ERSEM data to this resolution using the retrieved SS-DBEM data as a template.

First get the POLCOMS-ERSEM data.

In [6]:
xr_ensemble = xr.open_mfdataset('/var/data/CERES_fixed_depth/rcp45_2006-2099/201[3-7]/C3S-MCF_CERES_monthly.rcp45.*.*.nc')
xr_ensemble50 = xr.open_mfdataset('/var/data/CERES_fixed_depth/rcp45_2006-2099/2050/C3S-MCF_CERES_monthly.rcp45.*.*.nc')

# Extract the geographic and temporal range we need.

monthly_summary = xr_ensemble.sel(
                            lat=slice(48.4,63.5), 
                            lon=slice(-10.8,3.2), 
                            depth=0.5,
                            time=slice('2013-01-01','2017-12-31'))
monthly_summary50 = xr_ensemble50.sel(
                            lat=slice(48.4,63.5), 
                            lon=slice(-10.8,3.2), 
                            depth=0.5,
                            time=slice('2050-01-01','2050-12-31'))
monthly_summary = xr.concat((monthly_summary, monthly_summary50), dim='time').drop('depth')

The ERSEM data is provided at a monthly temporal resolution so we will aggregate it on a yearly timescale to match the SS-DBEM.

In [7]:
yearly_summary = monthly_summary[['thetao','so','ph']].groupby('time.year').mean('time')

Now get the SS-DBEM data.

We are only interested in specific species of fish for the two categories we wish to analyse (pelagic and demersal).

In [8]:
pelagic = ('herring', 'sardine', 'mackerel')
demersal = ('sole', 'plaice', 'turbot', 'atlantic_halibut', 'cod')

c = cdsapi.Client()

c.retrieve(
    'test-sis-operational-contract-c3s-442-lot2-pml-european-fisheries-fish-abundance',
    {
        'format':'zip',
        'model':'ss_dbem_polcoms',
        'variable':'species_catch',
        'experiment':'rcp4_5',
        'maximum_sustainable_yield':'0_6',
        'species': pelagic + demersal
    },
    'download.zip')

2019-07-22 08:32:44,556 INFO Sending request to https://cds-dev.copernicus-climate.eu/api/v2/resources/test-sis-operational-contract-c3s-442-lot2-pml-european-fisheries-fish-abundance
2019-07-22 08:32:50,245 INFO Request is completed
2019-07-22 08:32:50,248 INFO Downloading http://136.156.132.79/cache-compute-0000/cache/data1/dataset-test-sis-operational-contract-c3s-442-lot2-pml-european-fisheries-fish-abundance-7d39ac16-f719-480c-8e6c-9e5e8fac564c.zip to download.zip (4.1M)
2019-07-22 08:32:50,530 INFO Download rate 14.8M/s


Result(content_length=4349476,content_type=application/zip,location=http://136.156.132.79/cache-compute-0000/cache/data1/dataset-test-sis-operational-contract-c3s-442-lot2-pml-european-fisheries-fish-abundance-7d39ac16-f719-480c-8e6c-9e5e8fac564c.zip)

Pull out the time range and geographic area we are interested in.

In [9]:
xr_pel = {}
xr_pel50 = {}
xr_dem = {}
xr_dem50 = {}
with zip.ZipFile('download.zip') as myzip:
    for f in pelagic:
        fname = 'SS_DBEM_POLCOMS_fish_abundance-catch-rcp45-msy06-' + f + '-v0.1.nc'
        myzip.extract(fname)
        xr_pel[f] = xr.open_mfdataset(
                    fname).rename(
                        {'catch':f}).sel(
                            latitude=slice(48.4,63.5), 
                            longitude=slice(-10.8,3.2), 
                            time=slice('2013-01-01','2017-01-01'))
        xr_pel50[f] = xr.open_mfdataset(
                    fname).rename(
                        {'catch':f}).sel(
                            latitude=slice(48.4,63.5), 
                            longitude=slice(-10.8,3.2), 
                            time=slice('2050-01-01','2050-01-01'))
    for f in demersal:
        fname = 'SS_DBEM_POLCOMS_fish_abundance-catch-rcp45-msy06-' + f + '-v0.1.nc'
        myzip.extract(fname)
        xr_dem[f] = xr.open_mfdataset(
                    fname).rename(
                        {'catch':f}).sel(
                            latitude=slice(48.4,63.5), 
                            longitude=slice(-10.8,3.2), 
                            time=slice('2013-01-01','2017-01-01'))
        xr_dem50[f] = xr.open_mfdataset(
                    fname).rename(
                        {'catch':f}).sel(
                            latitude=slice(48.4,63.5), 
                            longitude=slice(-10.8,3.2), 
                            time=slice('2050-01-01','2050-01-01'))


We can use xESMF, https://xesmf.readthedocs.io/en/latest/index.html, to downsample the ERSEM data to the same resolution as the SS-DBEM data so download that now.

In [10]:
import xesmf as xe
regridder = xe.Regridder(yearly_summary, xr_pel['herring'].rename({'longitude': 'lon','latitude': 'lat'}), 'nearest_s2d')
#yearly_summary_small = regridder(yearly_summary)

Overwrite existing file: nearest_s2d_152x139_30x28.nc 
 You can set reuse_weights=True to save computing time.


**NOTE** Regridding may take a few minutes.

In [11]:
#xESMF only works with datarrays, it cannot do datasets at the moment so we have to split it up.
data_vars = {}
for name, data_var in yearly_summary.data_vars.items():
    #print (name, data_var)
    regridded = regridder(data_var)
    data_vars.update({name: regridded})
yearly_summary_small = xr.Dataset(data_vars)


/opt/conda/lib/python3.7/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


We can now map the data, starting with the average temperature.  

In [12]:
kdims = ['year', 'lon', 'lat']
vdims = [hv.Dimension(('ph','PH'), unit='')]
vdims = [hv.Dimension(('thetao','Temperature'), unit='deg C')]
temp_img = gv.Dataset(yearly_summary_small, 
                      kdims=kdims, 
                      vdims=[hv.Dimension(('thetao','Temperature'), unit='deg C')], 
                      crs=crs.PlateCarree()).to(gv.Image, 
                                     ['lon', 'lat']).options(cmap='plasma', 
                                                             fig_size=200) * gv.Feature(land_10m)
ph_img = gv.Dataset(yearly_summary_small, 
                      kdims=kdims, 
                      vdims=[hv.Dimension(('ph','PH'), unit='')], 
                      crs=crs.PlateCarree()).to(gv.Image, 
                                     ['lon', 'lat']).options(cmap='plasma', 
                                                             fig_size=200) * gv.Feature(land_10m)
sal_img = gv.Dataset(yearly_summary_small, 
                      kdims=kdims, 
                      vdims=[hv.Dimension(('so','Salinity'), unit='PSU')], 
                      crs=crs.PlateCarree()).to(gv.Image, 
                                     ['lon', 'lat']).options(cmap='plasma', 
                                                             fig_size=200) * gv.Feature(land_10m)
layout = hv.Layout([temp_img, ph_img, sal_img]).cols(3).opts(title='Mean surface temperature, PH and salinity from POLCOMS-ERSEM model. {dimensions}')
layout

:Layout
   .HoloMap.I   :HoloMap   [year]
      :Overlay
         .Image.I   :Image   [lon,lat]   (thetao)
         .Feature.I :Feature   [Longitude,Latitude]
   .HoloMap.II  :HoloMap   [year]
      :Overlay
         .Image.I   :Image   [lon,lat]   (ph)
         .Feature.I :Feature   [Longitude,Latitude]
   .HoloMap.III :HoloMap   [year]
      :Overlay
         .Image.I   :Image   [lon,lat]   (so)
         .Feature.I :Feature   [Longitude,Latitude]

**Figure 1** Mean surface temperature, PH and salinity from POLCOMS-ERSEM model. Use the slider to select the year of interest.

In [13]:
# Assemble the data into summary table.
df_yearly_temp = yearly_summary_small['thetao'].mean(dim=('lat','lon')).to_dataframe().rename(columns={'thetao':'Mean Temp (C)'})
df_yearly_temp['Min. Temp (C)'] = yearly_summary_small['thetao'].min(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Max. Temp (C)'] = yearly_summary_small['thetao'].max(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Mean pH']=yearly_summary_small['ph'].mean(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Max. pH']=yearly_summary_small['ph'].max(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Min pH']=yearly_summary_small['ph'].min(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Mean Salinity (PSU)']=yearly_summary_small['so'].mean(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Min. Salinity (PSU)']=yearly_summary_small['so'].max(dim=('lat','lon')).to_dataframe()
df_yearly_temp['Max. Salinity (PSU)']=yearly_summary_small['so'].min(dim=('lat','lon')).to_dataframe()

# and display it
df_yearly_temp.T

year,2013,2014,2015,2016,2017,2050
Mean Temp (C),11.553601,11.564861,11.266605,11.817730,11.602219,12.001348
Min. Temp (C),9.530485,9.523890,8.916962,9.301285,9.454273,9.561426
Max. Temp (C),14.289480,13.855420,14.146191,14.683505,14.197475,14.412483
Mean pH,8.174053,8.169248,8.160235,8.160442,8.180813,8.158527
Max. pH,8.308773,8.293538,8.280149,8.290803,8.317675,8.297204
Min pH,8.073436,8.070391,8.064793,8.069566,8.075026,8.045808
Mean Salinity (PSU),34.379545,34.410074,34.441434,34.435350,34.395536,34.372454
Min. Salinity (PSU),35.098230,35.109097,35.149589,35.054906,35.039650,35.175635
Max. Salinity (PSU),27.535751,27.058396,28.222330,28.154998,27.712827,27.488819


**Table 1** POLCOMS-ERSEM RCP 4.5 average, minimum and maximum per year for the whole of the UK EEZ

## b) Physical ecosystem service account
### Example application:  
Map potential catch for pelagic and demersal groups in the UK’s EEZ waters under climate change RCP4.5 at MSY 0.6 for the years 2013-2017 and 2050. Calculate the absolute potential catch in numbers of fish by species and group and present in tabular form.  

### Input data: 
SS-DBEM runs for RCP 4.5 at MSY 0.6 for the pelagic species: herring, sardines, and mackerel; and the demersal species: sole, plaice, turbot, halibut and cod in the domain 48.4° to 63.5°N and 10.8°W to 3.2°E for 2013-2017 and 2050.

### Output data: 
Potential yearly catch by pelagic and demersal group for 2013-2017 and 2050 mapped by gridded cell and aggregate totals for the whole EEZ by species and group.

### Python script: 
Extracts the yearly potential catch for herring, sardines, mackerel and adds them together to provide total potential catch for pelagics. The results are mapped by gridded cell for 2013-2017 and 2050 in the UK’s EEZ. Extracts the yearly potential catch for sole, plaice, turbot, halibut and cod and adds them together to provide total potential catch for demersals. The results are mapped by gridded cell for selected years. Total potential catch by fish species and sum total by group are extracted and presented in a table for all the years and downloadable as a csv file. 

Create some SS-DBEM datasets to use later.

In [14]:
pelagic_species = xr.merge(xr_pel.values()) 
pelagic_species50 = xr.merge(xr_pel50.values()) 
pelagic_species = xr.concat((pelagic_species, pelagic_species50), dim='time')

demersal_species = xr.merge(xr_dem.values()) 
demersal_species50 = xr.merge(xr_dem50.values()) 
demersal_species = xr.concat((demersal_species, demersal_species50), dim='time')

### Create summary tables for predicted fish catch.

In [15]:
pdt = pelagic_species.sum(dim=('latitude','longitude')).to_dataframe()
pdt['Total'] = pdt.sum(axis=1)
pelagic_species_summary = pdt
(pelagic_species_summary /1000).style.\
  set_caption('Potential pelargic fish catch (\'000 fish).').\
  format("{:0.0f}")

,herring,sardine,mackerel,Total
time,,,,
2013-01-01 00:00:00,616071,850485,92596,1559152
2014-01-01 00:00:00,706509,944115,110666,1761290
2015-01-01 00:00:00,942848,736354,111155,1790358
2016-01-01 00:00:00,352974,874986,67890,1295850
2017-01-01 00:00:00,779348,871791,118294,1769434
2050-01-01 00:00:00,583337,806485,96244,1486066


**Table2** Potential pelagic catch (millions of fish) present day and into the future

In [16]:
pdt = demersal_species.sum(dim=('latitude','longitude')).to_dataframe()
pdt['Total'] = pdt.sum(axis=1)
demersal_species_summary = pdt
(demersal_species_summary / 1000).style.\
  set_caption('Potential demersal fish catch (\'000 fish).').\
  format("{:0.0f}")

,sole,plaice,turbot,atlantic_halibut,cod,Total
time,,,,,,
2013-01-01 00:00:00,6694,118180,14194,10,23834,162912
2014-01-01 00:00:00,6901,118371,14091,10,23761,163134
2015-01-01 00:00:00,6927,119676,14205,10,23328,164147
2016-01-01 00:00:00,6449,120221,14184,10,23305,164169
2017-01-01 00:00:00,6470,119274,14031,10,23332,163117
2050-01-01 00:00:00,5839,103479,6605,8,20056,135988


**Table 3** Potential demersal catch (millions of fish) present day and into the future

### Create maps of predicted fish catch

In [17]:
kdims = ['time', 'longitude', 'latitude']
vdims = [f for f in pelagic]
vdims = ['total']

# Set the empty cells to a value of 0 so the summation works. Otherwise it will ignore the valid number for the other species.
pelagic_species = pelagic_species.fillna(0)
# Now add them all up.
pelagic_species = pelagic_species.assign({'total': pelagic_species['herring']+
                                          pelagic_species['sardine']+
                                          pelagic_species['mackerel']})
# Set any cells which are still zero back to NaN
pelagic_species['total'] = pelagic_species['total'].where(pelagic_species['total']>0)
ds = gv.Dataset(pelagic_species/1000, kdims=kdims, vdims=vdims, crs=crs.PlateCarree())
ds.to(gv.Image, ['longitude', 'latitude']).options(cmap='plasma', fig_size=200)* gv.Feature(land_10m) 

:HoloMap   [time]
   :Overlay
      .Image.I   :Image   [longitude,latitude]   (total)
      .Feature.I :Feature   [Longitude,Latitude]

**Figure 2**  Potential catch for pelagic species (\'000 fish) for 2013-2017,2050

In [18]:
kdims = ['time', 'longitude', 'latitude']
vdims = [f for f in demersal]
vdims = ['total']

# Set the empty cells to a value of 0 so the summation works. Otherwise it will ignore the valid number for the other species.
demersal_species = demersal_species.fillna(0)
# Now add them all up.
demersal_species = demersal_species.assign({'total': demersal_species['sole']+
                                          demersal_species['plaice']+
                                          demersal_species['turbot']+
                                          demersal_species['atlantic_halibut']+
                                          demersal_species['cod']})
# Set any cells which are still zero back to NaN
demersal_species['total'] = demersal_species['total'].where(demersal_species['total']>0)
ds = gv.Dataset(demersal_species/1000, kdims=kdims, vdims=vdims, crs=crs.PlateCarree())
ds.to(gv.Image, ['longitude', 'latitude']).options(cmap='plasma', fig_size=200)* gv.Feature(land_10m) 

:HoloMap   [time]
   :Overlay
      .Image.I   :Image   [longitude,latitude]   (total)
      .Feature.I :Feature   [Longitude,Latitude]

**Figure 3**  Potential catch for demersal species (\'000 fish) for 2013-2017,2050

## c) Monetary ecosystem service account
### Example application:  
Value the projected fish catch in 2050 and estimate asset value by pelagic and demersal group for 2013-2017 and 2050. 

### Input data: 
SS-DBEM runs for RCP 4.5 estimating potential catch in number of fish for the pelagic species: herring, sardines, and mackerel; and the demersal species: sole, plaice, turbot, halibut, cod. Data from the Marine Management Organisation Fisheries Statistics used to source actual landings in tonnes and value for 2013-2017 (MMO 2018).
### Output data: 
Value of potential catch for pelagic and demersal species in 2050 and asset value by pelagic and demersal group for 2013-2017 and 2050 in tabular form for comparison.

### Python script: 
Extracts the yearly potential catch in number of fish from the SS-DBEM runs for RCP 4.5 and MSY0.6 per species in the UK EEZ, in the domain 48.4° to 63.5°N and 10.8°W to 3.2°E for 2013-2017 and 2050. The total potential catch by species and group is presented as a table for each year and compared to published fisheries statistics in tonnes (MMO 2018) to estimate factors to convert number of fish to tonnes. The conversion factor is then averaged over the years 2013-2017 and used to estimate the tonnage of the projected catch in 2050. Price per tonnage is calculated from the value of landings and associated tonnage for 2013-2017 and used to calculated a 5-year average price used to value the potential catch in 2050 under the assumption of constant price. Finally, the value of the asset is estimated using the total value of catch by pelagic and demersal group.  
The equation, 
\begin{equation}
\mathbf{V}_{t} =
\frac
  {\mathbf{P}_{t}\times\mathbf{Q}_{t}}
  {r}
\end{equation} 

where $V_{t}$ = asset value at time t, P is price, Q is quantity and r is the discount rate is used with 3.5% discount employed. 


## Reference
MMO 2018. UK Sea Fisheries Statistics 2017. Available from https://www.gov.uk/government/statistics/uk-sea-fisheries-annual-statistics-report-2017. Accessed 02/07/19.  

In [19]:
# Set up the initial data frames based on the SS-DBEN download.
# We remove the index so it does not try and use it to join up the new data.
pelagic_species_landings = pd.DataFrame(pelagic_species_summary.reset_index()['time'])
demersal_species_landings = pd.DataFrame(demersal_species_summary.reset_index()['time'])

# Load the landing weights.
pelagic_species_landings['herring'] = pd.Series([93840.1102, 97683.4625999999, 93744.831,92247.2022,84057.8357])
pelagic_species_landings['sardine'] = pd.Series([ 3976.6224,3893.0669,4307.4921,9390.5689,7598.1182])
pelagic_species_landings['mackerel'] = pd.Series([163809.5026,287978.5913,247992.7932,217636.3938,226927.4646])

demersal_species_landings['sole'] = pd.Series([2270.2781,2317.73239999999,1940.2367,1962.4206,1803.0578])
demersal_species_landings['plaice'] = pd.Series([21226.1754,19143.3016,18875.5047,21223.2562,17772.0551])
demersal_species_landings['turbot'] = pd.Series([748.726199999998,840.288299999997,790.1392,872.992099999998,920.832699999999])
demersal_species_landings['atlantic_halibut'] = pd.Series([50.4345,36.3147,55.4166000000001,131.3183,205.0413])
demersal_species_landings['cod'] = pd.Series([29479.9872999999,30248.8647,28299.7008999999,34123.1511,38430.7233999999])

# Set the time column as the index to make the joins easier.
pelagic_species_landings = pelagic_species_landings.set_index('time')
demersal_species_landings = demersal_species_landings.set_index('time')

# Calculate the conversion factors for each year by dividing landings by predicted catch.
pelagic_conversion_factors = pelagic_species_landings / pelagic_species_summary
demersal_conversion_factors = demersal_species_landings / demersal_species_summary

In [38]:
# The 2050 value is the mean of the others.
pelagic_conversion_factors.iloc[5,:]=pelagic_conversion_factors[0:5].mean(axis=0)
demersal_conversion_factors.iloc[5,:]=demersal_conversion_factors[0:5].mean(axis=0)

pelagic_conversion_factors.drop('Total', axis=1, inplace=True)
demersal_conversion_factors.drop('Total', axis=1, inplace=True)

# We can now fill in the landings values for 2050 based on the predicted catch and our calculated conversion factor.
pelagic_species_landings.iloc[5,:] = (pelagic_conversion_factors * pelagic_species_summary).iloc[-1,1:]
demersal_species_landings.iloc[5,:] = (demersal_conversion_factors * demersal_species_summary).iloc[-1,1:]

### Fish landings

We can now produce tables for fish landing weights for our selected species based on the MMO data. This will be used to calculate conversion factors which allow us to estimate landing weights in the future based on predicted catch.

In [39]:
pelagic_species_landings.style.\
  set_caption('Pelagic fish landings (tonnes).').\
  format("{:0.0f}")

,herring,sardine,mackerel
time,,,
2013-01-01 00:00:00,93840,3977,163810
2014-01-01 00:00:00,97683,3893,287979
2015-01-01 00:00:00,93745,4307,247993
2016-01-01 00:00:00,92247,9391,217636
2017-01-01 00:00:00,84058,7598,226927
2050-01-01 00:00:00,88575,5500,225719


**Table 4** Pelagic fish landings (Source: MMO Fisheries Statistics 2017 for historical catches)

In [40]:
demersal_species_landings.style.\
  set_caption('Demersal fish landings (tonnes).').\
  format("{:0.0f}")

,sole,plaice,turbot,atlantic_halibut,cod
time,,,,,
2013-01-01 00:00:00,2270,21226,749,50,29480
2014-01-01 00:00:00,2318,19143,840,36,30249
2015-01-01 00:00:00,1940,18876,790,55,28300
2016-01-01 00:00:00,1962,21223,873,131,34123
2017-01-01 00:00:00,1803,17772,921,205,38431
2050-01-01 00:00:00,1796,17066,390,78,27414


**Table 5** Demersal fish landings (Source: MMO Fisheries Statistics 2017 for historical catches)

### Conversion factors

The conversion factors for each species are calculated by taking the landed weight and dividing by the predicted catch. The future conversion factor for 2050 is calculated as the mean of the 2013-2017 factors.

In [41]:
pelagic_conversion_factors

,herring,mackerel,sardine
time,,,
2013-01-01,0.000152,0.001769,0.000005
2014-01-01,0.000138,0.002602,0.000004
2015-01-01,0.000099,0.002231,0.000006
2016-01-01,0.000261,0.003206,0.000011
2017-01-01,0.000108,0.001918,0.000009
2050-01-01,0.000152,0.002345,0.000007


**Table 5** Conversion factors from number of fish to tonnes landed by species

In [42]:
demersal_conversion_factors

,atlantic_halibut,cod,plaice,sole,turbot
time,,,,,
2013-01-01,0.004951,0.001237,0.000180,0.000339,0.000053
2014-01-01,0.003578,0.001273,0.000162,0.000336,0.000060
2015-01-01,0.005558,0.001213,0.000158,0.000280,0.000056
2016-01-01,0.013196,0.001464,0.000177,0.000304,0.000062
2017-01-01,0.020793,0.001647,0.000149,0.000279,0.000066
2050-01-01,0.009615,0.001367,0.000165,0.000308,0.000059


**Table 6** Conversion factors from number of fish to tonnes landed by species

### Ecosystem services and assets

The price for each species is calculated by dividing the value of the fish landed by its tonnage and its value (MMO 2018). Valuing the 2050 projected catches is reliant on the assumptions of the evolution of the market price of the fish. In this demonstration, the prices for each of the fish species are assumed to remain constant into the future, an approach supported by the UN SEEA fisheries guidelines. The price to be used for 2050 valuation for each species is estimated using a five-year average to account for inter-annual variability.

Based on the recommendations of the HM Treasury, the discount rate of 3.5% is used.

In [43]:
fish_price_factors = {'herring':407.37, 
                           'sardine':312.82, 
                           'mackerel':817.78,
                           'sole':8188.73, 
                           'plaice':1203.13, 
                           'turbot':8020.80, 
                           'atlantic_halibut':8463.18, 
                           'cod':1759.96}

discount = 0.035

In [44]:
pelagic_species_value = pd.DataFrame()
for species in pelagic:
    pelagic_species_value[species] = pelagic_species_landings[species] * fish_price_factors[species]
pelagic_species_value['Total service value'] = pelagic_species_value.sum(axis=1)
pelagic_species_value['Total asset value'] = pelagic_species_value['Total service value'] / discount

In [45]:
(pelagic_species_value/1000000).style.\
  set_caption('Potential pelargic fish value (£m).').\
  format("{:0.2f}")

,herring,sardine,mackerel,Total service value,Total asset value
time,,,,,
2013-01-01 00:00:00,38.23,1.24,133.96,173.43,4955.19
2014-01-01 00:00:00,39.79,1.22,235.50,276.51,7900.41
2015-01-01 00:00:00,38.19,1.35,202.80,242.34,6924.00
2016-01-01 00:00:00,37.58,2.94,177.98,218.49,6242.71
2017-01-01 00:00:00,34.24,2.38,185.58,222.20,6348.46
2050-01-01 00:00:00,36.08,1.72,184.59,222.39,6354.05


**Table 7** Value of pelargic ecosystem services and assets

In [46]:
demersal_species_value = pd.DataFrame()
for species in demersal:
    demersal_species_value[species] = demersal_species_landings[species] * fish_price_factors[species]
demersal_species_value['Total service value'] = demersal_species_value.sum(axis=1)
demersal_species_value['Total asset value'] = demersal_species_value['Total service value'] / discount

In [47]:
(demersal_species_value/1000000).style.\
  set_caption('Potential demersal fish value (£m).').\
  format("{:0.2f}")

,sole,plaice,turbot,atlantic_halibut,cod,Total service value,Total asset value
time,,,,,,,
2013-01-01 00:00:00,18.59,25.54,6.01,0.43,51.88,102.44,2926.98
2014-01-01 00:00:00,18.98,23.03,6.74,0.31,53.24,102.30,2922.72
2015-01-01 00:00:00,15.89,22.71,6.34,0.47,49.81,95.21,2720.30
2016-01-01 00:00:00,16.07,25.53,7.00,1.11,60.06,109.77,3136.37
2017-01-01 00:00:00,14.76,21.38,7.39,1.74,67.64,112.90,3225.84
2050-01-01 00:00:00,14.71,20.53,3.13,0.66,48.25,87.27,2493.54


**Table 8** Value of demersal ecosystem services and assets